In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [3]:
%aimport model

In [4]:
import skorch

In [5]:
import numpy as np

In [ ]:
import keras

Using TensorFlow backend.


In [ ]:
import torch

In [ ]:
dataset = "fashion-mnist"

In [ ]:
dataset_config = {
    'size': 32,
    'channels': 1,
    'classes': 10
}

In [ ]:
model.Offnet(
    dataset,
    dataset_config['size'],
    dataset_config['channels'],
    dataset_config['classes'],
    total_block_number=6,
    widen_factor=5,
)

In [ ]:
net = skorch.net.NeuralNetClassifier(
    module=model.Offnet(
        dataset,
        dataset_config['size'],
        dataset_config['channels'],
        dataset_config['classes'],
        total_block_number=6,
        widen_factor=5,
    ),
    criterion=torch.nn.CrossEntropyLoss,
    batch_size=32,
    max_epochs=30,
    optimizer=torch.optim.Adam,
    callbacks=[
        ('checkpoint', skorch.callbacks.Checkpoint(target="models/untrainable_wrn{last_epoch[epoch]}.pt", monitor=None))
    ],
    device='cuda',
    train_split=None, # Workaround for https://github.com/dnouri/skorch/issues/218
    lr=1e-04,
)
net.initialize()

In [ ]:
def process(images):
    pad_images = np.pad(images, ((0, 0), (2, 2), (2, 2)), 'constant')
    number_of_images, height, width = pad_images.shape
    return pad_images.reshape(number_of_images, 1, height, width).astype(np.float32)  / 255.0

In [ ]:
(x_train_bytes, y_train_bytes), (x_test_bytes, y_test_bytes) = keras.datasets.fashion_mnist.load_data()
x_train = process(x_train_bytes)
x_test = process(x_test_bytes)
y_train = y_train_bytes.astype(np.int64)
y_test = y_test_bytes.astype(np.int64)

In [ ]:
net.fit_loop(x_train, y_train)

In [ ]:
prediction_test = net.predict(x_test)

In [ ]:
(prediction_test == y_test).mean()